### Transform Refunds Data
#### 1. Extract portion of refund_reason string using `SPLIT` function.

#### 2. Extract portion of refund_reason string using `REGEXP_EXTRACT` function.

#### 3. Extrat date and time from refund_timestamp.

#### 4. Write transformed data to Silver schema in hive metastore.

In [0]:
df_refunds_raw = spark.sql('select * from gizmobox.bronze.py_refunds')
display(df_refunds_raw)

#### 1. Extract portion of refund_reason string using `SPLIT` function.

In [0]:
from pyspark.sql import functions as F
df_refunds_transformed = df_refunds_raw.select(
                      'refund_id',
                      'payment_id',
                      F.date_format("refund_timestamp", "yyyy-MM-dd").cast('date').alias("refunda_date"),
                      F.date_format("refund_timestamp", "HH:MM:SS").alias("refund_time"),
                      'refund_timestamp',
                      'refund_amount',
                      F.split("refund_reason", ":")[0].alias("refund_reason"),
                      F.split("refund_reason", ":")[1].alias("refund_source")
)
display(df_refunds_transformed)

####  2. Extract portion of refund_reason string using `REGEXP_EXTRACT` function.

In [0]:
from pyspark.sql import functions as F
df_refunds_transformed_regexp = df_refunds_raw.select(
                      'refund_id',
                      'payment_id',
                      F.date_format("refund_timestamp", "yyyy-MM-dd").cast('date').alias("refunda_date"),
                      F.date_format("refund_timestamp", "HH:MM:SS").alias("refund_time"),
                      'refund_timestamp',
                      'refund_amount',
                      F.regexp_extract("refund_reason", "^([^:]+):", 1).alias("refund_reason"),
                      F.split("refund_reason", "^[^:]+:(.*)$", 1).alias("refund_source")
)
display(df_refunds_transformed_regexp)

####  4. Write transformed data to Silver schema in hive metastore.

In [0]:
df_refunds_transformed.writeTo("gizmobox.silver.py_refunds").createOrReplace()

In [0]:
%sql
SELECT * FROM gizmobox.silver.py_refunds;